# Setup

In [ ]:
from IPython.display import display

from brightway2 import *
import bw2analyzer as bwa
import brightway2 as bw
from bw2data.parameters import *
from lci_to_bw2 import *
from bw2data.project import ProjectManager
from bw2data.parameters import (ActivityParameter, DatabaseParameter,
                                ProjectParameter, Group)

import pandas as pd
import numpy as np

import math

import pathlib

import sqlite3

import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Directory with datasets:
ROOT_DIR = "files"

In [ ]:
# Define size of figure:
mpl.rcParams['figure.figsize'] = (16, 10)
pd.options.display.max_rows = 200

In [ ]:
# Define path to save figures:
path_img = os.path.abspath(os.path.join('outputs', 'IMG'))
if not os.path.exists(path_img):
    os.makedirs(path_img)
print(f'Images will be saved in {path_img}')

In [ ]:
# Define seaborn main parameters:
sns.set_style("ticks")
sns.color_palette("colorblind")
sns.set_context("paper", font_scale=1.5,
                rc={"axes.titlesize": 15, "lines.linewidth": 1.2,
                    "legend.fontsize": 10, "legend.title_fontsize": 10})

In [ ]:
# List of the available projects
bw.projects

In [ ]:
# New project or access an existing one:
bw.projects.set_current("LCA_Glazing")

# Where the current project is:
bw.projects.dir

**Abbreviation:**

- fg = flat glass
- inv = inventory
- act = activity
- norm = normalised impacts
- ia = impact assessment
- fu = functional unit
- igu = insulating glass unit

# List of Scenarios with their Parameters

Import the Excel file with the LCA scenarios (i.e. the values for the parameters to be used in BW2 and EnergyPlus):

In [ ]:
LCA_scenarios = pd.ExcelFile(os.path.join(ROOT_DIR, "LCA_scenarios.xlsx"))

Define a series of dataframes for each step of calculation:

In [ ]:
print("LCA_scenarios, sheet names = \n {}\n".format(LCA_scenarios.sheet_names))

In [ ]:
# Create dataframe for with scenarios for each step:
df_step1 = LCA_scenarios.parse('Step1').set_index('name')
df_step2 = LCA_scenarios.parse('Step2').set_index('name')
df_step3 = LCA_scenarios.parse('Step3').set_index('name')
df_step4 = LCA_scenarios.parse('Step4').set_index('name')
df_step5 = LCA_scenarios.parse('Step5').set_index('name')
df_step6 = LCA_scenarios.parse('Step6').set_index('name')
df_step7 = LCA_scenarios.parse('Step7').set_index('name')
df_step8 = LCA_scenarios.parse('Step8').set_index('name')
df_step9 = LCA_scenarios.parse('Step9').set_index('name')
df_step10 = LCA_scenarios.parse('Step10').set_index('name')
df_step11 = LCA_scenarios.parse('Step11').set_index('name')
df_step12 = LCA_scenarios.parse('Step12').set_index('name')
df_step13 = LCA_scenarios.parse('Step13').set_index('name')
df_step14 = LCA_scenarios.parse('Step14').set_index('name')
df_step15 = LCA_scenarios.parse('Step15').set_index('name')
df_step16 = LCA_scenarios.parse('Step16').set_index('name')
df_step17 = LCA_scenarios.parse('Step17').set_index('name')
df_step18 = LCA_scenarios.parse('Step18').set_index('name')
df_step19 = LCA_scenarios.parse('Step19').set_index('name')

# Import of LCA Databases

In [ ]:
# databases available in the current project:
bw.databases

## Ecoinvent and Biosphere 3

**Importing Biosphere 3:**

In [ ]:
# Importing elementary flows, LCIA methods and some other data
bw.bw2setup()

**Importing Ecoinvent 3.7, cut-off system model:**

For more information about the system models in ecoinvent 3, and especially the cut-off one, read [this](https://www.ecoinvent.org/database/system-models-in-ecoinvent-3/cut-off-system-model/allocation-cut-off-by-classification.html).

In [ ]:
# Import ecoinvent 3.7 cut-off
ei37cutdir = r"C:\Users\souvi\Documents\These\80_Calculations\06_LCA_SystemDiagrams\02_Dataset\ecoinvent 3.7_cutoff_ecoSpold02\datasets"

if 'ecoinvent 3.7 cut-off' in databases:
    print("Database has already been imported!")
else:
    ei37cut = bw.SingleOutputEcospold2Importer(
        ei37cutdir, 'ecoinvent 3.7 cut-off')
    ei37cut.apply_strategies()
    ei37cut.statistics()
    ei37cut.write_database()

## Excel datasets

<font color='red'>Do we need to import the Excel datasets?<font>

In [ ]:
import_exldb = True

**Importing the Excel dataset relating to aluminium production:**

In [ ]:
if import_exldb:
    imp = bw.ExcelImporter(os.path.join(ROOT_DIR, "LCI_Alu.xlsx"))
    imp.apply_strategies()
    imp.match_database(fields=('name', 'unit', 'location'))
    imp.match_database("ecoinvent 3.7 cut-off",
                       fields=('name', 'unit', 'location', 'input'))
    imp.statistics()

    # Checking whether the import went as expected.
    # Creation of an Excel sheet with process data:
    imp.write_excel()

    # Write the data to a database to save it:
    imp.write_database()

**Importing the Excel dataset relating to IGUs:**

In [ ]:
if import_exldb:
    imp = bw.ExcelImporter(os.path.join(ROOT_DIR, "LCI_IGU.xlsx"))
    imp.apply_strategies()
    imp.match_database(fields=('name', 'unit', 'location'))
    imp.match_database("ecoinvent 3.7 cut-off",
                       fields=('name', 'unit', 'location'))
    imp.match_database("exldb_alu",
                       fields=('name', 'unit', 'location', 'input'))
    imp.statistics()

    # Checking whether the import went as expected.
    # Creation of an Excel sheet with process data:
    imp.write_excel()

    # Add the project-level parameters:
    imp.write_project_parameters()

    # Write the data to a database to save it:
    imp.write_database()

**Importing the Excel dataset relating to double glazing w/ different types of spacers:**

In [ ]:
if import_exldb:
    imp = bw.ExcelImporter(os.path.join(ROOT_DIR, "LCI_Spacers.xlsx"))
    imp.apply_strategies()
    imp.match_database(fields=('name', 'unit', 'location'))
    imp.match_database("ecoinvent 3.7 cut-off",
                       fields=('name', 'unit', 'location'))
    imp.match_database("exldb_igu",
                       fields=('name', 'unit', 'location', 'input'))
    imp.statistics()

    # Checking whether the import went as expected.
    # Creation of an Excel sheet with process data:
    imp.write_excel()

    # Write the data to a database to save it:
    imp.write_database()

**Importing the Excel dataset relating to curtain wall end-of-life phase:**

In [ ]:
if import_exldb:
    imp = bw.ExcelImporter(os.path.join(ROOT_DIR, "LCI_CW_EoL.xlsx"))
    imp.apply_strategies()
    imp.match_database(fields=('name', 'unit', 'location'))
    imp.match_database("ecoinvent 3.7 cut-off",
                       fields=('name', 'unit', 'location'))
    imp.match_database("exldb_cw",
                       fields=('name', 'unit', 'location', 'input'))
    imp.statistics()

    # Checking whether the import went as expected.
    # Creation of an Excel sheet with process data:
    imp.write_excel()

    # Write the data to a database to save it:
    imp.write_database()

**Importing the Excel dataset relating to curtain wall production and use:**

In [ ]:
if import_exldb:
    imp = bw.ExcelImporter(os.path.join(ROOT_DIR, "LCI_CW.xlsx"))
    imp.apply_strategies()
    imp.match_database(fields=('name', 'unit', 'location'))
    imp.match_database("ecoinvent 3.7 cut-off",
                       fields=('name', 'unit', 'location'))
    imp.match_database("exldb_igu",
                       fields=('name', 'unit', 'location', 'input'))
    imp.match_database("exldb_alu",
                       fields=('name', 'unit', 'location', 'input'))
    imp.match_database("exldb_cw_eol",
                       fields=('name', 'unit', 'location', 'input'))
    imp.statistics()

    # Checking whether the import went as expected.
    # Creation of an Excel sheet with process data:
    imp.write_excel()

    # Write the data to a database to save it:
    imp.write_database()

**Checking if the imports went well:**

List databases:

In [ ]:
bw.databases

Checking Excel database:

**Deleting a database, if needed:**

## Navigate Databases

**Navigate Databases, assigning a variable to each one:**

In [ ]:
eib3db = bw.Database('biosphere3')

eicutdb = bw.Database('ecoinvent 3.7 cut-off')

exldb_alu = bw.Database('exldb_alu')
exldb_igu = bw.Database('exldb_igu')
exldb_cw = bw.Database('exldb_cw')
exldb_spacers = bw.Database('exldb_spacers')
exldb_cw_eol = bw.Database('exldb_cw_eol')

# Define the Parameters

## Overview

**Checking the total number of parameters:**

In [ ]:
len(parameters)

Look at the parameter directly:

In [ ]:
print("Project parameters:")
for p in ProjectParameter.select():
    print(p.name, ":", round(p.amount, 2))

print("------")
print("Database parameters:")
for p in DatabaseParameter.select():
    print(p.database, " > ", p.name, ":", round(p.amount, 2))

print("------")
print("Activity parameters:")
for p in ActivityParameter.select():
    if "dummy" not in p.name:
        print(p.name, ":", round(p.amount, 2))

## Activate Parameters

**We have now to tell bw to activate the exchanges and their formulas:**

First, we tell the parameter code that these exchanges have formulas:

In [ ]:
# Add exchanges with formulas from activity to group:
for act in exldb_igu:
    parameters.add_exchanges_to_group("igu_param_group", act)

In [ ]:
ls_act_data_cw_eol = []
n_code = 0
for obj in DatabaseParameter.select().where(
        DatabaseParameter.database == "exldb_cw_eol"):
    ls_act_data_cw_eol.append({'name': obj.name, 'amount': obj.amount,
                               'formula': obj.formula, 'database': obj.database,
                              'code': "p_eol_"+str(n_code)})
    n_code += 1

parameters.new_activity_parameters(
    ls_act_data_cw_eol, "cw_eol_param_group", overwrite=True)

In [ ]:
for act in exldb_cw_eol:
    parameters.add_exchanges_to_group("cw_eol_param_group", act)

In [ ]:
ls_act_data_cw = []
n_code = 0
for obj in DatabaseParameter.select().where(
        DatabaseParameter.database == "exldb_cw"):
    ls_act_data_cw.append({'name': obj.name, 'amount': obj.amount,
                           'formula': obj.formula, 'database': obj.database,
                           'code': "p_"+str(n_code)})
    n_code += 1

parameters.new_activity_parameters(
    ls_act_data_cw, "cw_use_param_group", overwrite=True)

In [ ]:
for act in exldb_cw:
    parameters.add_exchanges_to_group("cw_use_param_group", act)

Then we calculate the "activated" formula values:

In [ ]:
ActivityParameter.recalculate_exchanges("igu_param_group")
ActivityParameter.recalculate_exchanges("cw_use_param_group")
ActivityParameter.recalculate_exchanges("cw_eol_param_group")

**Delete the parameters:**

# LCIA Methods

List of available LCIA methods:

Define methods:

In [ ]:
method_ilcd_gwp = (
    'ILCD 2.0 2018 midpoint', 'climate change', 'climate change total')

In [ ]:
methods_small = [
    ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change total'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater ecotoxicity'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality',
     'freshwater and terrestrial acidification'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater eutrophication'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'terrestrial eutrophication'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'ozone layer depletion'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'photochemical ozone creation'),
    ('ILCD 2.0 2018 midpoint', 'resources', 'fossils'),
    ('ILCD 2.0 2018 midpoint', 'resources', 'land use')
]

In [ ]:
methods_full = [
    ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change total'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater ecotoxicity'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality',
     'freshwater and terrestrial acidification'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater eutrophication'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'marine eutrophication'),
    ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'terrestrial eutrophication'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'non-carcinogenic effects'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'carcinogenic effects'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'ionising radiation'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'ozone layer depletion'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'photochemical ozone creation'),
    ('ILCD 2.0 2018 midpoint', 'human health', 'respiratory effects, inorganics'),
    ('ILCD 2.0 2018 midpoint', 'resources', 'minerals and metals'),
    ('ILCD 2.0 2018 midpoint', 'resources', 'dissipated water'),
    ('ILCD 2.0 2018 midpoint', 'resources', 'fossils'),
    ('ILCD 2.0 2018 midpoint', 'resources', 'land use')
]

# LCA of Glass Sheets and IGUs from Cradle to Gate

## Flat glass production

Define the <font color='red'> functional unit </font> per flat glass (fg) type:

In [ ]:
inv_fg_unsorted = [act for act in exldb_igu
                   if 'market for flat glass' in act['name']
                   ]

inv_fg = sorted(inv_fg_unsorted,
                key=lambda k: k['name'])

inv_fg

In [ ]:
names_fg_processed = ['market for laminated safety glass',
                      'market for tempered safety glass',
                      'market for smart glass'
                      ]

inv_fg_processed_unsorted = [act for act in exldb_igu
                             for n in names_fg_processed
                             if n in act['name']
                             and "glazing" not in act['name']
                             ]

inv_fg_processed = sorted(inv_fg_processed_unsorted,
                          key=lambda k: k['name'])

inv_fg_processed

In [ ]:
# functional unit; 25kg of glass to obtain a thickness of 10mm for 1m²:
fu_fg = 25
# functional unit; 1m²:
fu_fg_processed = 1

impacts_fg = []

for act in inv_fg:
    lca = bw.LCA({act: fu_fg})
    lca.lci()
    for method in methods_small:
        lca.switch_method(method)
        lca.lcia()
        impacts_fg.append((act["name"], act["location"],
                           method[1].title(), lca.score,
                           bw.methods.get(method).get('unit')))

for act in inv_fg_processed:
    lca = bw.LCA({act: fu_fg_processed})
    lca.lci()
    for method in methods_small:
        lca.switch_method(method)
        lca.lcia()
        impacts_fg.append((act["name"], act["location"],
                           method[1].title(), lca.score,
                           bw.methods.get(method).get('unit')))

# impacts_fg

In [ ]:
df_impacts_fg = pd.DataFrame(impacts_fg, columns=["Name", "Location", "Method",
                                                  "Score", "Unit"])

df_impacts_fg = pd.pivot_table(df_impacts_fg, index=["Name"],
                               columns=["Method", "Unit"],
                               values="Score")

In [ ]:
df_impacts_fg = df_impacts_fg.sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True)

df_impacts_fg.round(2)

In [ ]:
df_impacts_fg = df_impacts_fg.reset_index()
df_impacts_fg['Name'] = (df_impacts_fg['Name']
                         .str.replace('market for ', ''))

df_impacts_fg = df_impacts_fg.set_index('Name')

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))

df_impacts_wo_smartg = df_impacts_fg.drop("smart glass", axis=0)

sns.barplot(data=df_impacts_wo_smartg,
            x=("Climate Change", "kg CO2-Eq"),
            y=df_impacts_wo_smartg.index,
            color="white", edgecolor=".2", linewidth=1.5)

ax.yaxis.label.set_visible(False)
ax.grid(which='major', axis='x', linestyle=':', linewidth=1)

fig.suptitle(
    'GWP of different types of glass panes, cradle-to-gate', y=1.1)
sns.despine(left=True, offset=5)
plt.show()

In [ ]:
df_norm_impacts_wo_smartg = (
    df_impacts_wo_smartg / df_impacts_wo_smartg.max()
)
df_norm_impacts_wo_smartg.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3.5))

sns.barplot(data=df_impacts_fg,
            x=("Climate Change", "kg CO2-Eq"),
            y=df_impacts_fg.index,
            color="white", edgecolor=".2", linewidth=1.5)

ax.yaxis.label.set_visible(False)
ax.grid(which='major', axis='x', linestyle=':', linewidth=1)

fig.suptitle(
    'GWP of different types of glass panes, cradle-to-gate')
sns.despine(left=True, offset=5)
plt.show()

In [ ]:
df_norm_impacts_fg = df_impacts_fg / df_impacts_fg.max()
df_norm_impacts_fg.round(2)

**Same calculation, but using the MultiLCA class:**

In [ ]:
# functional unit; 25kg of glass to obtain a thickness of 10mm for 1m²:
fu_fg = 25
# functional unit; 1m²:
fu_fg_processed = 1

mlca_syst_fg = []

for act in inv_fg:
    mlca_syst_fg.append({act.key: fu_fg})

for act in inv_fg_processed:
    mlca_syst_fg.append({act.key: fu_fg_processed})

mlca_syst_fg

In [ ]:
bw.calculation_setups['calculation_setup'] = {'inv': mlca_syst_fg,
                                              'ia': methods_small}

mlca = bw.MultiLCA('calculation_setup')
df_impacts_mlca_fg = pd.DataFrame(data=mlca.results, columns=mlca.methods)

In [ ]:
activities = [(get_activity(key), amount)
              for dct in mlca.func_units
              for key, amount in dct.items()
              ]

df_fu = pd.DataFrame([(x['name'], x['database'], x['code'],
                       x['location'], x['unit'], y)
                      for x, y in activities],
                     columns=('Database', 'Code', 'Name',
                              'Location', 'Unit', 'Amount')
                     )

df_impacts_mlca_fg_expanded = pd.concat(
    [df_fu, df_impacts_mlca_fg], axis=1
)

In [ ]:
df_impacts_mlca_fg_expanded = (
    df_impacts_mlca_fg_expanded.set_index("Name")
    .drop(["Database", "Code", "Location"], axis=1).round(2)
)

In [ ]:
df_impacts_mlca_fg_expanded.sort_values(
    by=[('ILCD 2.0 2018 midpoint', 'climate change', 'climate change total')],
    ascending=True)

## Analysis of the Impact of Different Types of Spacers, Sealants and Insulating Gases, Cradle-to-Gate

**Comparative analysis of different spacers:**

In [ ]:
inv_spacers = [act for act in bw.Database("exldb_spacers")
               if 'krypton' not in act['name']
               and 'xenon' not in act['name']
               and 'air' not in act['name']]

fu_spacers = [{igu: 1} for igu in inv_spacers]

In [ ]:
impacts_spacers = []

for igu in inv_spacers:
    lca = bw.LCA({igu: 1})
    lca.lci()
    for method in methods_small:
        lca.switch_method(method)
        lca.lcia()
        impacts_spacers.append((igu["name"], igu["location"],
                                method[1].title(), lca.score,
                                bw.methods.get(method).get('unit')))

# impacts_spacers

In [ ]:
df_impacts_spacers = pd.DataFrame(
    impacts_spacers,
    columns=["Name", "Location", "Method", "Score", "Unit"]
)

df_impacts_spacers = pd.pivot_table(
    df_impacts_spacers, index=["Name"],
    columns=["Method", "Unit"], values="Score"
)

In [ ]:
df_impacts_spacers = df_impacts_spacers.sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True
)

df_impacts_spacers = df_impacts_spacers.reset_index()
df_impacts_spacers['Name'] = (df_impacts_spacers['Name']
                              .str.replace('double glazing production, ', '')
                              .str.replace(', argon', ''))

df_impacts_spacers = df_impacts_spacers.set_index('Name')

In [ ]:
df_impacts_spacers.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))

sns.barplot(data=df_impacts_spacers,
            x=("Climate Change", "kg CO2-Eq"),
            y=df_impacts_spacers.index,
            color="white", edgecolor=".2", linewidth=1.5)

ax.yaxis.label.set_visible(False)
ax.grid(which='major', axis='x', linestyle=':', linewidth=1)

fig.suptitle(
    'GWP of double glazing according to spacer types, cradle-to-gate')
sns.despine(left=True, offset=5)
plt.show()

In [ ]:
df_norm_impacts_spacers = df_impacts_spacers / df_impacts_spacers.max()
df_norm_impacts_spacers.round(2).sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True)

**Comparative analysis of different infill gases:**

In [ ]:
inv_gas = [act for act in bw.Database("exldb_spacers")
           if 'thermally broken aluminium' in act['name']
           ]

fu_gas = [{igu: 1} for igu in inv_gas]
fu_gas

In [ ]:
impacts_gas = []

for igu in inv_gas:
    lca = bw.LCA({igu: 1})
    lca.lci()
    for method in methods_small:
        lca.switch_method(method)
        lca.lcia()
        impacts_gas.append((igu["name"], igu["location"],
                            method[1].title(), lca.score,
                            bw.methods.get(method).get('unit')))

# impacts_gas

In [ ]:
df_impacts_gas = pd.DataFrame(
    impacts_gas,
    columns=["Name", "Location", "Method", "Score", "Unit"]
)

df_impacts_gas = pd.pivot_table(
    df_impacts_gas, index=["Name"],
    columns=["Method", "Unit"], values="Score"
)

In [ ]:
df_impacts_gas = df_impacts_gas.sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True
)

df_impacts_gas = df_impacts_gas.reset_index()
df_impacts_gas['Name'] = (df_impacts_gas['Name']
                          .str.replace('double glazing production, ', '')
                          )

df_impacts_gas = df_impacts_gas.set_index('Name')

In [ ]:
df_impacts_gas.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 2))

sns.barplot(data=df_impacts_gas,
            x=("Climate Change", "kg CO2-Eq"),
            y=df_impacts_gas.index,
            color="white", edgecolor=".2", linewidth=1.5)

ax.yaxis.label.set_visible(False)
ax.grid(which='major', axis='x', linestyle=':', linewidth=1)

fig.suptitle(
    'GWP of double glazing according to insulating gas types, cradle-to-gate', y=1.1)
sns.despine(left=True, offset=5)
plt.show()

In [ ]:
df_norm_impacts_gas = df_impacts_gas / df_impacts_gas.max()
df_norm_impacts_gas.round(2).sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True)

## Comparative LCA of Different Designs for IGU, Cradle-to-Gate

In [ ]:
inv_igus = [act for act in bw.Database("exldb_igu")
            if 'market' in act['name']
            and ('glazing' in act['name']
            or 'vacuum' in act['name'])
            ]

fu_igus = [{igu: 1} for igu in inv_igus]

fu_igus

In [ ]:
impacts_igus = []

for igu in inv_igus:
    lca = bw.LCA({igu: 1})
    lca.lci()
    for method in methods_small:
        lca.switch_method(method)
        lca.lcia()
        impacts_igus.append((igu["name"], igu["location"],
                             method[1].title(), lca.score,
                             bw.methods.get(method).get('unit')))

# impacts_igus

In [ ]:
df_impacts_igus = pd.DataFrame(
    impacts_igus,
    columns=["Name", "Location", "Method", "Score", "Unit"]
)

df_impacts_igus = pd.pivot_table(
    df_impacts_igus, index=["Name"],
    columns=["Method", "Unit"], values="Score"
)

In [ ]:
df_impacts_igus = df_impacts_igus.sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True
)

df_impacts_igus = df_impacts_igus.reset_index()
df_impacts_igus['Name'] = (df_impacts_igus['Name']
                           .str.replace('market for ', ''))

df_impacts_igus = df_impacts_igus.set_index('Name')

In [ ]:
df_impacts_igus.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

df_impacts_igus_wo_smartg = df_impacts_igus.drop(
    "smart glass, double glazing", axis=0
)

sns.barplot(data=df_impacts_igus_wo_smartg,
            x=("Climate Change", "kg CO2-Eq"),
            y=df_impacts_igus_wo_smartg.index,
            color="white", edgecolor=".2", linewidth=1.5)

ax.yaxis.label.set_visible(False)
ax.grid(which='major', axis='x', linestyle=':', linewidth=1)

fig.suptitle(
    'GWP of different design of IGUs, cradle-to-gate')
sns.despine(left=True, offset=5)
plt.show()

In [ ]:
df_norm_impacts_igus_wo_smartg = (
    df_impacts_igus_wo_smartg / df_impacts_igus_wo_smartg.max()
)

df_norm_impacts_igus_wo_smartg.round(2).sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6.5))

sns.barplot(data=df_impacts_igus,
            x=("Climate Change", "kg CO2-Eq"),
            y=df_impacts_igus.index,
            color="white", edgecolor=".2", linewidth=1.5)

ax.yaxis.label.set_visible(False)
ax.grid(which='major', axis='x', linestyle=':', linewidth=1)

fig.suptitle(
    'GWP of different design of IGUs, cradle-to-gate')
sns.despine(left=True, offset=5)
plt.show()

In [ ]:
df_norm_impacts_igus = df_impacts_igus / df_impacts_igus.max()
df_norm_impacts_igus.round(2).sort_values(
    ("Climate Change", "kg CO2-Eq"), ascending=True)

# Import Results from the BEM

## Recover data saved in csv files from previous simulations:

To read directly the csv file and work from them instead of relaunching the energy simulation (Please note that this does not prevent the simulations from being run again. To do this, change the run_all boolean variable above. If the simulations are rerun, the new results will overwrite the old ones saved in the csv files):

Dataframe with all energy usage data per simulation run:

Retrieve the data from the csv. The simulation_postprocess function will merge the new results by overwriting the old ones, or simply add the new ones if they did not exist yet.

In [ ]:
# Open the df_end_use_allsteps from the csv file:
# Avoid re-running energy simulations (time consuming):
if os.path.isfile('outputs\steps_dir\df_end_use_allsteps.csv'):
    df_end_use_allsteps_csv = (
        pd.read_csv('outputs\steps_dir\df_end_use_allsteps.csv'))
    df_end_use_allsteps_csv = df_end_use_allsteps_csv.pivot_table(
        values='0', index=['EndUse'], columns=['Run name', 'FuelType'])

    df_end_use_allsteps = df_end_use_allsteps_csv

A function to recover df_step dataframes saved as csv, i.e. DataFrame with the main assumptions and results (natural gas and electricity) specific to each simulation run:

To assess the indirect impact of glazing replacement on energy use in the building, the natural gas and electricity use results for each scenario are subtracted by the initial scenario, where the exact same glazing is kept. 

In [ ]:
def recover_df_step(n_step, df_step):
    """
    If a df_step.csv exists, recover it as a dataframe wich replace 
    the one currently in use in the notebook.
    Avoid re-running energy simulation (time consuming).

    Parameters
    ----------
    n_step: number of the step
    df_step: a dataframe. followed by a number (e.g. step4), 
    identify the step with simulation runs and main results

    Returns
    -------
    df_step: update with csv data or exactly the same as the one in the input

    """

    # Does the csv exist
    # and check if the existing df_step includes simulation results:
    if os.path.isfile(f"outputs\steps_dir\df_step"+str(n_step)+".csv"):
        df_step = (
            pd.read_csv(f"outputs\steps_dir\df_step"+str(n_step)+".csv")
            .set_index(['name']))

        print("df_step ", n_step, "updated with csv data")
    else:
        print("existing df_step ", n_step, "kept in place")

    # Subtraction of energy use by that in an initial scenario
    if not df_step.loc[df_step["glazing"] == "dg_init_bronze"].empty:
        i_gas = float(
            df_step.loc[df_step["glazing"] == "dg_init_bronze", "natural_gas"])
        i_elec = float(
            df_step.loc[df_step["glazing"] == "dg_init_bronze", "elec_use"])

        df_step["natural_gas"] = (df_step["natural_gas"] - i_gas)
        df_step["elec_use"] = (df_step["elec_use"] - i_elec)

    else:
        print("DG_init not in step ", n_step,
              "! energy use not substracted by dg_init!")

    return df_step

## Recover the df_step dataframes:

In [ ]:
df_step1 = recover_df_step(1, df_step1)
df_step2 = recover_df_step(2, df_step2)
df_step3 = recover_df_step(3, df_step3)
df_step4 = recover_df_step(4, df_step4)
df_step5 = recover_df_step(5, df_step5)
df_step6 = recover_df_step(6, df_step6)
df_step7 = recover_df_step(7, df_step7)
df_step8 = recover_df_step(8, df_step8)
df_step9 = recover_df_step(9, df_step9)
df_step10 = recover_df_step(10, df_step10)
df_step11 = recover_df_step(11, df_step11)
df_step12 = recover_df_step(12, df_step12)
df_step13 = recover_df_step(13, df_step13)
df_step14 = recover_df_step(14, df_step14)
df_step15 = recover_df_step(15, df_step15)
df_step16 = recover_df_step(16, df_step16)
df_step17 = recover_df_step(17, df_step17)
df_step18 = recover_df_step(18, df_step18)
df_step19 = recover_df_step(19, df_step19)

# Impact Assessment of the Whole Life Cycle of Curtain Walls Retrofitting

## Setup of the LCA

First, we define the activity of dismantling, and thus disposal, of the existing curtain wall:

In [ ]:
out_old_cw = exldb_cw.get('dismantling_cw_old_dg')
# Check:
print('My activity is:\n', out_old_cw)

Then we define the production activity of the new curtain wall:

In [ ]:
prod_cw = exldb_cw.get('production_cw')
# Check:
print('My activity is:\n', prod_cw)

We define the use phase activity (not linked to production):

In [ ]:
use_bldg_w_cw = exldb_cw.get('use_glazed_office_bldg')
# Check:
print('My activity is:\n', use_bldg_w_cw)

We define a use phase activity integrating the production phase:

In [ ]:
prod_and_use_cw = exldb_cw.get('use_cw')
# Check:
print('My activity is:\n', prod_and_use_cw)

We define a maintenance activity:

In [ ]:
repair_cw = exldb_cw.get('maintenance_cw')
# Check:
print('My activity is:\n', repair_cw)

And finally, the end-of-life activity:

In [ ]:
eol_cw = exldb_cw.get('eol_cw')
# Check:
print('My activity is:\n', eol_cw)

Define the lifespan (years):

In [ ]:
lifespan = [p.amount for p in DatabaseParameter.select()
            if p.name == 'param_lifespan'][0]

## Functions to Perform the LCAs

Write a function to perform the LCA according to activities and parameter sets:

In [ ]:
def lca_cw_gwp(df_step, act, fu):
    """
    Perform a simple lca for different scenarios 
    according to parameters defined in df_step

    Parameters
    ----------
    df_step: DataFrame with list of parameters and their values
    act: activity to assess
    fu: functional unit

    Returns
    -------
    ls_results: list of values for IPCC GWP
    """

    # To save results:
    ls_results = []

    # Define a dataframe only with bw2 parameters:
    df_param = df_step.drop(['glazing',
                             'heating_setpoint',
                             'cooling_setpoint'], axis=1)

    # Convert dataframe in a numpy array:
    val_np = df_param.to_numpy()

    n_scenario = 0

    for v in val_np:
        name_scenario = df_param.index[n_scenario]
        n_scenario += 1

        for i in range(len(df_param.columns)):
            # Change parameters according to column.index:
            param_name = df_param.columns[i]
            n = df_param.columns.get_loc(param_name)

            (ActivityParameter.update(amount=v[n])
             .where(ActivityParameter.name == f'param_{param_name}').execute())

        ActivityParameter.recalculate_exchanges("cw_use_param_group")
        ActivityParameter.recalculate_exchanges("cw_eol_param_group")

        lca = LCA({act: fu}, method_ilcd_gwp)
        lca.lci()
        lca.lcia()
        ls_results.append({'run': name_scenario, 'result': lca.score})

    return ls_results

Write a function to perform a multi_method LCA, with small_methods, according to activities and parameter sets:

In [ ]:
# Reminder of the reduced list of methods:
methods_small

In [ ]:
def lca_cw_mlca_small(df_step, act, fu):
    """
    Perform a multi-method lca for different scenarios 
    according to parameters defined in df_step.
    Methods= ReCiPe: GWP100, ODPinf, PMFP, POFP

    Parameters
    ----------
    df_step: DataFrame with list of parameters and their values
    act: activity to assess
    fu: functional unit

    Returns
    -------
    ls_mlca_small_results: list of values
    """

    # To save results:
    ls_mlca_small_results = []

    # Define a dataframe only with bw2 parameters:
    df_param = df_step.drop(['glazing', 'heating_setpoint',
                             'cooling_setpoint'], axis=1)

    # Convert dataframe in a numpy array:
    val_np = df_param.to_numpy()

    n_scenario = 0

    for v in val_np:
        name_scenario = df_param.index[n_scenario]
        n_scenario += 1

        for i in range(len(df_param.columns)):
            # Change parameters according to column.index:
            param_name = df_param.columns[i]
            n = df_param.columns.get_loc(param_name)

            (ActivityParameter.update(amount=v[n])
             .where(ActivityParameter.name == f'param_{param_name}').execute())

        ActivityParameter.recalculate_exchanges("cw_use_param_group")
        ActivityParameter.recalculate_exchanges("cw_eol_param_group")

        lca = LCA({act: fu})
        lca.lci()

        for method in methods_small:
            lca.switch_method(method)
            lca.lcia()
            ls_mlca_small_results.append((name_scenario,
                                          method[2].title(), lca.score,
                                          bw.methods.get(method).get('unit')))

    return ls_mlca_small_results

Write a function to perform a multi_method LCA, with full_methods, according to activities and parameter sets:

In [ ]:
# Reminder of the full list of methods:
methods_full

In [ ]:
def lca_cw_mlca_full(df_step, act, fu):
    """
    Perform a multi-method lca for different scenarios 
    according to parameters defined in df_step.
    Methods= ReCiPe: GWP100, ODPinf, PMFP, POFP

    Parameters
    ----------
    df_step: DataFrame with list of parameters and their values
    act: activity to assess
    fu: functional unit

    Returns
    -------
    ls_mlca_full_results: list of values
    """

    # To save results:
    ls_mlca_full_results = []

    # Define a dataframe only with bw2 parameters:
    df_param = df_step.drop(['glazing', 'heating_setpoint',
                             'cooling_setpoint'], axis=1)

    # Convert dataframe in a numpy array:
    val_np = df_param.to_numpy()

    n_scenario = 0

    for v in val_np:
        name_scenario = df_param.index[n_scenario]
        n_scenario += 1

        for i in range(len(df_param.columns)):
            # Change parameters according to column.index:
            param_name = df_param.columns[i]
            n = df_param.columns.get_loc(param_name)

            (ActivityParameter.update(amount=v[n])
             .where(ActivityParameter.name == f'param_{param_name}').execute())

        ActivityParameter.recalculate_exchanges("cw_use_param_group")
        ActivityParameter.recalculate_exchanges("cw_eol_param_group")

        lca = LCA({act: fu})
        lca.lci()

        for method in methods_full:
            lca.switch_method(method)
            lca.lcia()
            ls_mlca_full_results.append((name_scenario,
                                         method[2].title(), lca.score,
                                         bw.methods.get(method).get('unit')))

    return ls_mlca_full_results

A little function to transform a list of mlca_results in a DataFrame:

In [ ]:
def ls_to_df_mlca(ls):
    """
    A little function to transform the ls_mlca_results 
    in a readable DataFrame

    Parameters
    ----------
    ls: the list

    Returns
    -------
    df: the DataFrame
    """

    # DataFrame to then work w/ results:
    df = pd.DataFrame(ls,
                      columns=["Name",
                               "Impact Category",
                               "Score",
                               "Unit"
                               ]
                      )

    df = pd.pivot_table(df,
                        index=["Name"],
                        columns=["Impact Category",
                                 "Unit"
                                 ],
                        values="Score"
                        )

    return df

A function to save mlca_full_results in a DataFrame, for each simulation run and LCA phase:

In [ ]:
# Initialise a DataFrame:
df_mlca_full_raw_results = pd.DataFrame()

In [ ]:
def ls_to_df_mlca_full(step, ls, act, df_results):
    """
    A function to append a list of mlca results in a DataFrame, 
        with values organised per simulation run (index), 
        and LCA phase (columns).

    Parameters
    ----------
    step: correspond to the batch of simulation runs: step_1, 2...
    ls: the list of results.
    df_results: a DataFrame where LCA results will be saved.
    act: activity for which the LCA has been done.

    Returns
    -------
    df_results
    """

    # New DataFrame from list of results:
    df_temp = pd.DataFrame(ls,
                           columns=["Name",
                                    "Impact Category",
                                    "Score",
                                    "Unit"
                                    ]
                           )

    # Add information regarding the step:
    df_temp["Step"] = step
    # Add information regarding the LCA phase:
    df_temp["LCA Phase"] = str(act["name"])

    # Pivot the DataFrame:
    df_temp = pd.pivot_table(df_temp,
                             index=["Step",
                                    "Name"
                                    ],
                             columns=["LCA Phase",
                                      "Impact Category",
                                      "Unit"
                                      ],
                             values="Score"
                             )
    # Merge with existing results:
    if df_results.empty:
        df_results = df_temp
        print("empty, df_results replaced")
    else:
        # Merge by columns_to_use:
        df_results = pd.concat(
            [df_results, df_temp[~df_temp.index.isin(df_results.index)]]
        )
        df_results.update(df_temp)

    return df_results

A function to conduct a mLCA per activity and save the mlca_full_results in a DataFrame, for each simulation run and each LCA phase:

In [ ]:
def full_lca_to_df(step, df_step, df_results, fu):
    """
    A function to conduct and LCA (w/ function: lca_cw_mlca_full)
        and append a list of mlca results in a DataFrame 
        (w/ function: ls_to_df_mlca_full), 
        with values organised per simulation run (index),
        and LCA phase (columns).

    Parameters
    ----------
    step: correspond to the batch of simulation runs: step_1, 2...
    df_step: DataFrame with list of parameters and their values

    Returns
    -------
    df_results: a DataFrame where LCA results are saved,
        simulation run as index, LCA phase and impact indicators as columns.
    """

    for act in [prod_cw, use_bldg_w_cw, repair_cw, eol_cw]:
        ls = lca_cw_mlca_full(df_step, act, fu)
        df_results = ls_to_df_mlca_full(step, ls, act, df_results)

    return df_results

## Environmental Analysis, Step-by-Step

### Life Cycle Impact Assessment

Full life cycle assessment, ILCD methods, from cradle-to-grave, for each simulation run:

<font color="red">Performing the impact assessment or loading the csv file where previous results are stored?<font>

In [ ]:
# Do we perform the LCIA?
calc_lcia = False

Be patient, takes time to calculate !

In [ ]:
if calc_lcia:
    # LCIA calculation:
    ls_df_step = [
        df_step1, df_step2, df_step3, df_step4, df_step5,
        df_step6, df_step7, df_step8, df_step9, df_step10,
        df_step11, df_step12, df_step13, df_step14, df_step15,
        df_step16, df_step17, df_step18, df_step19
    ]

    n = 1
    for df in ls_df_step:
        step = "step_"+str(n)
        df_mlca_full_raw_results = full_lca_to_df(step, df,
                                                  df_mlca_full_raw_results, 1)
        n += 1

    # Save df_mlca_full_raw_results to csv:
    df_mlca_full_raw_results.unstack([0, 1]).to_csv(
        'outputs\steps_dir\df_mlca_full_raw_results.csv', index=True)

else:
    # Open the csv file, to avoid recalculating the impacts:
    if os.path.isfile('outputs\steps_dir\df_mlca_full_raw_results.csv'):
        df_mlca_full_raw_results = (
            pd.read_csv('outputs\steps_dir\df_mlca_full_raw_results.csv')
        )
        df_mlca_full_raw_results = df_mlca_full_raw_results.pivot_table(
            values='0',
            index=['Step', 'Name'],
            columns=['LCA Phase', 'Impact Category', 'Unit']
        )

In [ ]:
df_mlca_full_raw_results

### Navigate the LCIA Result DataFrame

List of impact categories:

In [ ]:
df_ilcd_methods = pd.DataFrame()
n = 0
ls_n = []
ls_ic = []
ls_ic_details = []
ls_u = []
for method in methods_full:
    ls_n.append(n + 1)
    ls_ic.append(method[1].title())
    ls_ic_details.append(method[2].title())
    ls_u.append(bw.methods.get(method).get('unit'))
    n += 1

df_ilcd_methods["Impact Category"] = ls_ic
df_ilcd_methods["Sub-Category"] = ls_ic_details
df_ilcd_methods["Unit"] = ls_u
df_ilcd_methods["#"] = ls_n

df_ilcd_methods = df_ilcd_methods.set_index(["Impact Category", "#"])
df_ilcd_methods

Select the impact category to display:

In [ ]:
# Define the rank of the impact category (#):
n = 4

Select the step of analysis, with the batch of simulation runs:

In [ ]:
step = "step_4"

In [ ]:
ic = df_ilcd_methods.xs(n, level=1)["Sub-Category"][0]
df_mlca_full_raw_results.loc[step].xs(ic, axis=1,
                                      level=1, drop_level=False)

### Disposal of the Existing Curtain Wall

In [ ]:
ls_mlca_oldcw_results = []

lca = bw.LCA({out_old_cw: 1})
lca.lci()
for method in methods_full:
    lca.switch_method(method)
    lca.lcia()
    ls_mlca_oldcw_results.append((method[2].title(), lca.score,
                                  bw.methods.get(method).get('unit')))

ls_mlca_oldcw_results

In [ ]:
df_mlca_oldcw_results = pd.DataFrame(ls_mlca_oldcw_results,
                                     columns=["Impact Category",
                                              "Score",
                                              "Unit"]
                                     )

df_mlca_oldcw_results = pd.pivot_table(df_mlca_oldcw_results,
                                       columns=["Impact Category", "Unit"],
                                       values="Score"
                                       )

df_mlca_oldcw_results

# Post-Processing and Data Analysis

In [ ]:
df_lca_lifespan = pd.DataFrame(
    {'Year': np.arange(lifespan+3),
     'Step': 'ref',
     'Scenario': 'no_retrofit',
     'Impact Category': 'All',
     'Unit': 'None',
     'Score': 0
     }
)

df_lca_lifespan = df_lca_lifespan.pivot(index='Year',
                                        columns=['Step',
                                                 'Scenario',
                                                 'Impact Category',
                                                 'Unit'
                                                 ],
                                        values='Score'
                                        )

In [ ]:
df_mlca_full_raw_results = df_mlca_full_raw_results.reset_index(level=0)

# Defining the columns, one for each simulation run:
for run in df_mlca_full_raw_results.index:
    n_step = df_mlca_full_raw_results["Step"].loc[run]
    for method in methods_full:
        ic = method[2].title()
        u = bw.methods.get(method).get('unit')
        # define a new column:
        df_lca_lifespan[n_step, run, ic, u] = 0

df_lca_lifespan = df_lca_lifespan.drop("ref", axis=1)

In [ ]:
# LCIA over the 40 years of the service life of the curtain wall:
for step, run, ic, u in df_lca_lifespan.columns:
    # First phase of the LCA, disposal of the existing curtain wall:
    df_lca_lifespan.loc[0][step, run, ic, u] = (
        df_mlca_oldcw_results[ic, u]
    )

    # Second phase, production/construction of the new curtain wall:
    df_lca_lifespan.loc[1][step, run, ic, u] = (
        df_lca_lifespan.loc[0][step, run, ic, u] +
        df_mlca_full_raw_results.loc[run][
            "curtain wall, production", ic, u
        ]
    )

    # Third phase, use of the curtain wall, indirect energy use impacts:
    for y in range(2, 42):
        df_lca_lifespan.loc[y][step, run, ic, u] = (
            df_lca_lifespan.loc[y-1][step, run, ic, u] +
            df_mlca_full_raw_results.loc[run][
                "use of glazed office building, hvac and lighting", ic, u
            ]
        )
        if (y == 12 or y == 22 or y == 32):
            # Third phase, impacts relating to maintenance, every 10y:
            df_lca_lifespan.loc[y][step, run, ic, u] += (
                df_mlca_full_raw_results.loc[run][
                    "curtain wall, maintenance", ic, u
                ]
            )

    # Last phase, end-of-life of the new curtain wall:
    df_lca_lifespan.loc[42][step, run, ic, u] = (
        df_lca_lifespan.loc[41][step, run, ic, u] +
        df_mlca_full_raw_results.loc[run][
            "curtain wall, end of life", ic, u
        ]
    )

Post-processing the LCA results to define the trajectory 2020 > 2050, with climate change:

In [ ]:
# Code for the simulations run in 2020 which corresponds
# to the same configuration as the one in 2050, 
# step_14 > step_4; step_16 > step_10; step_18 > step_11:
run_2050_2020 = {
    "n_a_2126_dg_init_cc2050": "d_a_2126_dg_init_vav_sd",
    "n_b_2126_dg0_cc2050": "d_b_2126_dg0_vav_sd",
    "n_c_2126_dg4_cc2050": "d_h_2126_dg4_vav_sd",
    "n_d_2126_dg5_cc2050": "d_i_2126_dg5_vav_sd",
    "n_e_2126_dg6_cc2050": "d_j_2126_dg6_vav_sd",
    "n_f_2126_tg4_cc2050": "d_n_2126_tg4_vav_sd",
    "n_g_2126_tg5_cc2050": "d_o_2126_tg5_vav_sd",
    "n_h_2126_tg6_cc2050": "d_p_2126_tg6_vav_sd",
    "p_a_2124_dg_init_cc2050": "j_a_2124_dg_init_sd",
    "p_b_2124_dg0_cc2050": "j_b_2124_dg0_sd",
    "p_c_2124_dg4_cc2050": "j_c_2124_dg4_sd",
    "p_d_2124_dg5_cc2050": "j_d_2124_dg5_sd",
    "p_e_2124_dg6_cc2050": "j_e_2124_dg6_sd",
    "p_f_2124_tg4_cc2050": "j_f_2124_tg4_sd",
    "p_g_2124_tg5_cc2050": "j_g_2124_tg5_sd",
    "p_h_2124_tg6_cc2050": "j_h_2124_tg6_sd",
    "r_a_1927_dg_init_cc2050": "k_a_1927_dg_init_tc",
    "r_b_1927_dg0_cc2050": "k_b_1927_dg0_tc",
    "r_c_1927_dg4_cc2050": "k_c_1927_dg4_tc",
    "r_d_1927_dg5_cc2050": "k_d_1927_dg5_tc",
    "r_e_1927_dg6_cc2050": "k_e_1927_dg6_tc",
    "r_f_1927_tg4_cc2050": "k_f_1927_tg4_tc",
    "r_g_1927_tg5_cc2050": "k_g_1927_tg5_tc",
    "r_h_1927_tg6_cc2050": "k_h_1927_tg6_tc"
}

In [ ]:
# Corresponding simulations 2050 > 2020:
steps_2050_2020 = {"step_14": "step_4",
                   "step_16": "step_10",
                   "step_18": "step_11"
                   }

In [ ]:
# LCIA over 40years for 2050 climate change scenario:
# 2022 corresponds to y = 2
for step, run, ic, u in df_lca_lifespan.columns:
    # First step, heating 21, cooling 26:
    if step in steps_2050_2020.keys():
        # Modification of the first year in use, i.e. 2022 and not 2050:
        df_lca_lifespan.loc[2][step, run, ic, u] = (
            df_lca_lifespan.loc[2][steps_2050_2020[step], 
                                   run_2050_2020[run], 
                                   ic, 
                                   u
                                  ]
        )

        # Delete data between 2022 and 2050, i.e. y=30:
        for y in range(3, 30):
            df_lca_lifespan.loc[y][step, run, ic, u] = np.nan

        # Idem passed 2050, i.e. y=30:
        for y in range(31, 43):
            df_lca_lifespan.loc[y][step, run, ic, u] = np.nan

        # Interpolate between 2020 and 2050, and the 10 years after:
        df_lca_lifespan[step, run, ic, u].interpolate(
            method='linear',
            limit_direction='forward',
            axis=0
        )

        # Last phase, end-of-life of the new curtain wall:
        df_lca_lifespan.loc[42][step, run, ic, u] += (
            df_mlca_full_raw_results.loc[run][
                "curtain wall, end of life", ic, u
            ]
        )

In [ ]:
# Rebuild df_mlca_full_raw_results:
df_mlca_full_raw_results = (
    df_mlca_full_raw_results.reset_index().set_index(["Step", "Name"])
)

In [ ]:
df_lca_lifespan["step_16"].xs(
    "Climate Change Total", axis=1, level=1, drop_level=False)

In [ ]:
# Plot the evolution of the GWP per year:
fig, ax = plt.subplots(figsize=(16, 9))

for glazing in df_step1.index:
    if 'dg' in glazing:
        if 'dg_init' in glazing:
            sns.lineplot(data=df_step1_gwp.reset_index(),
                         x='Year', y=('1', '1_1926_dg_init'),
                         color='firebrick',
                         ax=ax)

        else:
            sns.lineplot(data=df_step1_gwp.reset_index(),
                         x='Year', y=('1', glazing),
                         color='steelblue',
                         ax=ax)
    if 'tg' in glazing:
        sns.lineplot(data=df_step1_gwp.reset_index(),
                     x='Year', y=('1', glazing),
                     color='forestgreen',
                     ax=ax)

ax.fill_between(np.array(df_step1_gwp.index),
                np.array(df_step1_gwp['1', '1_1926_dg3']),
                np.array(df_step1_gwp['1', '1_1926_dg6']), alpha=0.25)

ax.axhline(y=0, c='black', linestyle=':', linewidth=0.75)
ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

ax.yaxis.label.set_visible(False)
ax.set_xlim(0, 45)
ax.set_ylim(-500, 1000)

fig.suptitle('')
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot the evolution of the GWP per year:
fig, ax = plt.subplots(figsize=(16, 9))

for glazing in df_step1.index:
    if 'dg_init' in glazing:
        sns.lineplot(data=df_step1_gwp.reset_index(),
                     x='Year', y=('1', '1_1926_dg_init'),
                     color='firebrick',
                     ax=ax)

    if 'tg' in glazing:
        sns.lineplot(data=df_step1_gwp.reset_index(),
                     x='Year', y=('1', glazing),
                     color='forestgreen',
                     ax=ax)

ax.fill_between(np.array(df_step1_gwp.index),
                np.array(df_step1_gwp['1', '1_1926_dg3']),
                np.array(df_step1_gwp['1', '1_1926_dg6']), alpha=0.10)

ax.axhline(y=0, c='black', linestyle=':', linewidth=0.75)
ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

ax.yaxis.label.set_visible(False)
ax.set_xlim(0, 45)
ax.set_ylim(-500, 1000)

fig.suptitle('')
sns.despine(offset=5)
plt.show()

In [ ]:
# Plot the evolution of the GWP per year:
fig, ax = plt.subplots(figsize=(16, 9))

for glazing in df_step1.index:
    if 'dg_init' in glazing:
        sns.lineplot(data=df_step1_gwp.reset_index(),
                     x='Year', y=('1', '1_1926_dg_init'),
                     color='firebrick',
                     ax=ax)

    if 'sg' in glazing:
        sns.lineplot(data=df_step1_gwp.reset_index(),
                     x='Year', y=('1', glazing),
                     color='darkorange',
                     ax=ax)

ax.fill_between(np.array(df_step1_gwp.index),
                np.array(df_step1_gwp['1', '1_1926_dg3']),
                np.array(df_step1_gwp['1', '1_1926_dg6']), alpha=0.10)

ax.axhline(y=0, c='black', linestyle=':', linewidth=0.75)
ax.grid(which='major', axis='y', linestyle=':', linewidth=1)

ax.yaxis.label.set_visible(False)
ax.set_xlim(0, 45)
ax.set_ylim(-500, 1000)

fig.suptitle('')
sns.despine(offset=5)
plt.show()

# Energy Mix: Sensitivity Analysis

In [ ]:
# List of activities to change, in this case electricity markets:
locations = ["FR", "DE", "PL", "NL", "CH"]

act_name = "market for electricity, low voltage"

el_markets = [('ecoinvent 3.7 cut-off', act['code'])
              for act in eicutdb.search(act_name, limit=200)
              for location in locations
              if act_name in act['name'] and location in act['location']
              and "US-FRCC" not in act['location']
              and "US-SERC" not in act['location']]

# Remove "market for electricity, low voltage, label-certified" for CH:
el_markets.pop(5)

el_markets

In [ ]:
act_cw = exldb_cw.get('use_cw')

# Printing the exchanges
print('My activity is:\n', act_cw,
      '\n-------\nAnd its exchanges:\n-------')

for i in list(act_cw.exchanges()):
    print(i['type'])
    print(i)
    print(i['input'])
    print('-------')

In [ ]:
exc = list(act_cw.exchanges())[10]
exc

In [ ]:
# Make a copy of the activity, substitute the background process
# save and calculate:

cw_elec_results = []

for m in el_markets[0:2]:
    act_cw_copy = act_cw.copy()
    exc['input'] = m
    exc.save()
    lca = LCA({act_cw_copy: 1}, ('IPCC 2013', 'climate change', 'GWP 100a'))
    lca.lci()
    lca.lcia()
    cw_elec_results.append(lca.score)

In [ ]:
# Results for Belgium:
lca = LCA({act_cw: 1}, method_ilcd_gwp)
lca.lci()
lca.lcia()
be_cw_elec_result = lca.score

In [ ]:
be_cw_elec_result

In [ ]:
np.mean([(i/be_cw_elec_result) for i in cw_elec_results])

In [ ]:
pd.DataFrame(cw_elec_results).describe()

In [ ]:
plt.boxplot(cw_elec_results)
plt.ylabel(methods[('IPCC 2013', 'climate change', 'GWP 100a')]['unit'])
plt.xlabel('El Production')